In [ ]:
# downloaded from https://grouplens.org/datasets/movielens/25m/

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import time
from tqdm import tqdm
dbg = True
# from google.colab import drive
# drive.mount('/content/gdrive')
nrows = {True:100000, False:None}[dbg]
rating_df = pd.read_csv('ml-25m/ratings.csv', nrows=nrows)
tag_df = pd.read_csv('ml-25m/tags.csv', nrows=nrows)
movie_df = pd.read_csv('ml-25m/movies.csv')
rating_df['userId'] = rating_df['userId'] - 1
rating_df['movieId'] = rating_df['movieId'] - 1
tag_df['userId'] = tag_df['userId'] - 1
tag_df['movieId'] = tag_df['movieId'] - 1
movie_df['movieId'] = movie_df['movieId'] - 1
rating_df['movieId'] = rating_df['movieId'].map({v: i for i, v in enumerate(rating_df['movieId'].unique())})
mismatch = set(rating_df["movieId"]).difference(movie_df["movieId"])
rating_df = rating_df.loc[~rating_df['movieId'].isin(mismatch)]

movie_counts = rating_df['movieId'].value_counts()
popular_movies = movie_counts[(movie_counts > 30)].index
rating_df = rating_df.loc[rating_df['movieId'].isin(popular_movies)]
user_counts = rating_df['userId'].value_counts()
popular_users = user_counts[(user_counts > 30)].index
rating_df = rating_df.loc[rating_df['userId'].isin(popular_users)]

movie_df = movie_df.loc[movie_df['movieId'].isin(rating_df['movieId'].unique())]
movie_id_map = {v:k for k, v in enumerate(rating_df['movieId'].unique())}
movie_df['movieId'] = movie_df['movieId'].map(movie_id_map).astype(int)
rating_df['movieId'] = rating_df['movieId'].map(movie_id_map).astype(int)
user_id_map = {v:k for k, v in enumerate(rating_df['userId'].unique())}
rating_df['userId'] = rating_df['userId'].map(user_id_map).astype(int)
rating_df = rating_df.reset_index(drop=True)
movie_df = movie_df.reset_index(drop=True)
mismatch = set(movie_df["movieId"]).difference(rating_df["movieId"])
movie_df = movie_df.drop(mismatch)
num_users = rating_df['userId'].nunique()
num_movies = rating_df['movieId'].nunique()

In [11]:
movie_df

,movieId,title,genres
0,136,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,726,Jumanji (1995),Adventure|Children|Fantasy
2,731,Father of the Bride Part II (1995),Comedy
3,510,Heat (1995),Action|Crime|Thriller
4,515,GoldenEye (1995),Action|Adventure|Thriller
...,...,...,...
768,767,Women on the Verge of a Nervous Breakdown (Muj...,Comedy|Drama
769,768,Any Given Sunday (1999),Drama
770,769,"Loves of Carmen, The (1948)",Drama
771,771,Turtle Diary (1985),Comedy|Drama|Romance


In [ ]:
f(userid, movieid) => 

In [9]:
rating_df

,userId,movieId,rating,timestamp
0,0,0,3.5,1141415820
1,0,1,0.5,1141417130
2,0,2,5.0,1141416589
3,0,3,4.0,1141415790
4,0,4,4.5,1141415643
...,...,...,...,...
51352,511,166,3.5,1184073818
51353,511,62,3.5,1184013545
51354,511,758,3.5,1184017555
51355,511,589,3.0,1184073493


In [13]:
train_rows=np.random.choice(rating_df.shape[0], int(0.8*rating_df.shape[0]), replace=False)
ratings_train = rating_df.loc[train_rows]
test_rows = set(range(rating_df.shape[0])).difference(set(train_rows))
ratings_test = rating_df.loc[test_rows]

In [ ]:
(user vector * movie vector) => rating

In [18]:
minrating, maxrating = rating_df['rating'].describe().loc[['min', 'max']]

In [25]:
tfph = tf.compat.v1.placeholder
tfvar = tf.compat.v1.get_variable
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
emb_size = 32

user_vec = tfvar('uservec', shape = (num_users, emb_size), dtype = tf.float32)
movie_vec = tfvar('movievec', shape = (num_movies, emb_size), dtype = tf.float32)
user_bias_vec = tfvar('userbiasvec', shape = (num_users), dtype = tf.float32)
movie_bias_vec = tfvar('moviebiasvec', shape = (num_movies), dtype = tf.float32)

user_ph = tfph(tf.int32, shape = (None))
movie_ph = tfph(tf.int32, shape = (None))
rating_ph = tfph(tf.float32, shape = (None))

# batch size, emb_size
user_emb = tf.gather(user_vec, user_ph)
movie_emb = tf.gather(movie_vec, movie_ph)

# batch size
user_bias_emb = tf.gather(user_bias_vec, user_ph)
movie_bias_emb = tf.gather(movie_bias_vec, movie_ph)

score_raw = tf.reduce_sum(user_emb * movie_emb, axis=1) + user_bias_emb + movie_bias_emb
score = score_raw * (maxrating- minrating) + minrating

mse = tf.reduce_sum(tf.square(score - rating_ph))